In [1]:
%matplotlib inline
from IPython.display import Markdown, display
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import math

In [2]:
density_of_air = 1.293
area_of_prop_blade = 3.334e-3
propeller_slip = 0.2
propeller_pitch = 10.67 / 100
# max_air_speed = 20

delta_t = 0.01

In [3]:
quadratic_equation = np.array([ 0, 0.03789803, -0.00061863])
def get_poly(x):
    x2 = quadratic_equation[2] * (x**2)
    x1 = quadratic_equation[1] * x
    x0 = quadratic_equation[0] * x
    return np.array([x0, x1, x2])

def solve_quadratic(a, b, c, y):
    c_prime = c - y
    discriminant = b**2 - 4*a*c_prime

    if discriminant > 0:
        x1 = (-b + math.sqrt(discriminant)) / (2 * a)
        x2 = (-b - math.sqrt(discriminant)) / (2 * a)
        return (x1, x2)
    elif discriminant == 0:
        x = -b / (2 * a)
        return (x,)
    else:
        # No real roots
        return ()

def get_wind_speed_from_rps(rps):
    return rps * propeller_pitch * (1-propeller_slip)

def get_coeff_of_lift(force, wind_speed):
    return 2 * force / (density_of_air * area_of_prop_blade * (wind_speed**2))

def get_fin_angle_from_coef_of_lift(cl):
    max_angle = 30
    roots = solve_quadratic(quadratic_equation[2], quadratic_equation[1], quadratic_equation[0], cl)
    if len(roots) == 2:
        if roots[0] >= 0 and roots[0] <= max_angle and roots[1] >= 0 and roots[1] <= max_angle:
            angle = min(roots[0], min[roots[1]])
        elif roots[0] >= 0 and roots[0] <= max_angle:
            angle = roots[0]
        elif roots[1] >= 0 and roots[1] <= max_angle:
            angle = roots[1]
        else:
            # need to work
            angle = 0
    elif len(roots) == 1:
        if roots[0] < 0:
            angle = 0
        elif roots[0] > max_angle:
            angle = max_angle
    else:
        angle = max_angle

    return angle

def get_fin_angle_from_force(force, rps):
    wind_speed = get_wind_speed_from_rps(rps)
    cl = get_coeff_of_lift(abs(force), wind_speed)
    fin_angle = get_fin_angle_from_coef_of_lift(cl)
    return fin_angle * (force / abs(force))

In [4]:
factor = 0.0017696
# factor = 0.005
prop_speed = 140
target_angle = 0
current_angle = 30


kp = 6
ki = 0
kd = 0

pid_i = 0
prev_err = 0

err = target_angle - current_angle
pid_p = kp * err
if abs(err) < 5:
    pid_i += ki * err

pid_d = kd * (err - prev_err) / delta_t
prev_err = err

sum = pid_p + pid_i + pid_d

get_fin_angle_from_force(sum * 0.5 * factor, prop_speed)

np.float64(-20.53679297557252)